In [164]:
import pandas as pd
from bs4 import BeautifulSoup
from requests.exceptions import RequestException
from contextlib import closing
from requests import get
import unicodedata
import re
import json
import logging

In [6]:
def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None
    
def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)

In [150]:
url = r'https://www.hemnet.se/salda/bostader?location_ids%5B%5D=17755&item_types%5B%5D=villa&item_types%5B%5D=radhus&item_types%5B%5D=bostadsratt&rooms_min=2.5&selling_price_min=2000000&selling_price_max=4000000&sold_age=all'

raw_html = simple_get(url = url)
html = BeautifulSoup(raw_html, 'html.parser')

In [90]:
def find_numbers(text):
    
    regexp = re.compile('(\d+)')
    
    results = regexp.findall(text)
    price_str = ''
    for result in results:
        price_str+=result
    return float(price_str)
    

In [157]:
def load_house_data(url):
    
    data = {}
    
    raw_html = simple_get(url = url)
    html_ = BeautifulSoup(raw_html, 'html.parser')
    
    item = html_.find(class_ = 'sold-property__price')
    text = item.find(class_ = 'sold-property__price-value').text
    text_ = unicodedata.normalize('NFKD',text)
    data['price'] = find_numbers(text = text_)
    
    item = html_.find(class_ = 'sold-property__details')
    attribute_names = []
    for attribute in item.find_all(class_ = 'sold-property__attribute'):
        attribute_names.append(attribute.text)

    attributes = {}

    for name,value_item in zip(attribute_names,item.find_all(class_ = 'sold-property__attribute-value')):
        value_str = unicodedata.normalize('NFKD',value_item.text)
        #value = find_numbers(text = value_str)
        attributes[name] = value_str
        
    data.update(attributes)
    
    map_item = html_.find(class_ = 'sold-property__map')
    map_data = json.loads(map_item['data-initial-data'])

    listing = map_data['listing']
    data['id'] = listing['id']
    data['coordinate'] = listing['coordinate']
    data['type'] = listing['type']
    data['address'] = listing['address']
    data['map_url'] = map_data['map_url']
    data['url'] = listing['url']
    
    data = pd.Series(data)
    data.name = data['id']
    
    return data

In [173]:
house_data.to_csv

ERROR:root:Skipping house
Traceback (most recent call last):
  File "<ipython-input-173-0ac8724e2624>", line 17, in <module>
    data = load_house_data(url = item_link_contaier['href'])
  File "<ipython-input-157-7bf0f9b6129c>", line 28, in load_house_data
    map_data = json.loads(map_item['data-initial-data'])
TypeError: 'NoneType' object is not subscriptable
ERROR:root:Skipping house
Traceback (most recent call last):
  File "<ipython-input-173-0ac8724e2624>", line 17, in <module>
    data = load_house_data(url = item_link_contaier['href'])
  File "<ipython-input-157-7bf0f9b6129c>", line 28, in load_house_data
    map_data = json.loads(map_item['data-initial-data'])
TypeError: 'NoneType' object is not subscriptable
ERROR:root:Skipping house
Traceback (most recent call last):
  File "<ipython-input-173-0ac8724e2624>", line 17, in <module>
    data = load_house_data(url = item_link_contaier['href'])
  File "<ipython-input-157-7bf0f9b6129c>", line 28, in load_house_data
    map_data = 

In [176]:
house_data.describe()

,id,price
count,11997.000000,1.199700e+04
mean,746968.338751,2.919749e+06
std,114428.849637,5.771279e+05
min,351317.000000,2.000000e+06
25%,651325.000000,2.440000e+06
50%,753989.000000,2.850000e+06
75%,843379.000000,3.400000e+06
max,938587.000000,4.000000e+06


In [177]:
house_data.to_csv('house_data.csv')